In [2]:
import pandas as pd

# Läser in fliken "Veckodata Riket" från datafilen "fhm_covid19" .
df_covid19 = pd.read_excel('Data/fhm_covid19.xlsx', sheet_name='Veckodata Riket')

# En initial dataanalys av "Veckodata Riket"
print(df_covid19.info())
print(df_covid19.describe())
print(df_covid19.value_counts())
print(df_covid19.head())
print(df_covid19.columns)
print(df_covid19.index)

# Slår ihop kolumnerna "år" och "veckonummer" till en "Vecka"-kolumn.
df_covid19['Vecka'] = df_covid19['år'].astype(str) + "v" + df_covid19['veckonummer'].astype(str)

print(df_covid19.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142 entries, 0 to 141
Data columns (total 13 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   år                              142 non-null    int64  
 1   veckonummer                     142 non-null    int64  
 2   Antal_fall_vecka                142 non-null    int64  
 3   Antal_fall_100000inv_vecka      142 non-null    int64  
 4   Antal_fall_100000inv_14dagar    142 non-null    int64  
 5   Kum_antal_fall                  142 non-null    int64  
 6   Kum_fall_100000inv              142 non-null    int64  
 7   Antal_nyaintensivvårdade_vecka  142 non-null    int64  
 8   Kum_antal_intensivvårdade       142 non-null    int64  
 9   Antal_avlidna_vecka             142 non-null    int64  
 10  Antal_avlidna_milj_inv_vecka    142 non-null    float64
 11  Kum_antal_avlidna               142 non-null    int64  
 12  Kum_antal_avlidna_milj_inv      142 